In [ ]:
# Import libraries - added for attention and GRU
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GRU, Dense, Dropout, Bidirectional, Input, Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2  # IMPROVEMENT: Add L2 regularization

from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  # IMPROVEMENT: More metrics
from sklearn.model_selection import TimeSeriesSplit  # IMPROVEMENT: For better validation

# Custom Attention Layer
class SimpleAttention(Layer):
    def __init__(self):
        super(SimpleAttention, self).__init__()
        self.dense = Dense(1)

    def call(self, inputs):
        attention_weights = tf.nn.softmax(self.dense(inputs), axis=1)
        return tf.reduce_sum(inputs * attention_weights, axis=1)

# Configuration
plt.style.use('default')
warnings.filterwarnings('ignore')
np.random.seed(42)
tf.random.set_seed(42)

print("✅ Libraries imported - Enhanced for Experiment 7!")

# Submission system remains the same
def save_submission(predictions, experiment_name, test_index, notes=""):
    os.makedirs('submissions', exist_ok=True)
    
    submission = pd.DataFrame({
        'row ID': test_index.strftime('%Y-%m-%d %H:%M:%S'),
        'pm2.5': np.round(predictions).astype(int)
    })
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'submissions/{timestamp}_{experiment_name}.csv'
    
    submission.to_csv(filename, index=False)
    
    print(f"✅ Submission saved: {filename}")
    print(f"📊 Predictions - Min: {predictions.min():.1f}, Max: {predictions.max():.1f}, Mean: {predictions.mean():.1f}")
    print(f"📝 Notes: {notes}")
    
    return filename, submission

print("✅ Submission system ready - targeting <3400 RMSE!")

# Data loading remains the same
print("📂 Loading datasets...")
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

train.set_index('datetime', inplace=True)
test.set_index('datetime', inplace=True)

print("🔧 Handling missing values...")
train_clean = train.fillna(method='ffill').fillna(method='bfill').interpolate()
test_clean = test.fillna(method='ffill').fillna(method='bfill').interpolate()

# IMPROVED feature engineering - Add EMAs and more interactions
def create_enhanced_features(df, target_col='pm2.5'):
    df_enhanced = create_focused_features(df, target_col)  # Start with Notebook 6 features
    
    print("🔧 Adding advanced features (Experiment 7)...")
    # IMPROVEMENT: Exponential Moving Averages for smoother trends
    if target_col in df_enhanced.columns:
        for span in [12, 24, 48]:
            df_enhanced[f'pm2.5_ema_{span}'] = df_enhanced[target_col].ewm(span=span).mean()
    
    # IMPROVEMENT: More weather interactions
    df_enhanced['temp_humidity_interact'] = df_enhanced['TEMP'] * df_enhanced['humidity_proxy']
    df_enhanced['wind_temp_interact'] = df_enhanced['Iws'] * df_enhanced['TEMP']
    
    # Drop any new NaNs
    df_enhanced = df_enhanced.fillna(method='bfill').fillna(0)
    
    return df_enhanced

def create_test_features_enhanced(df):
    return create_enhanced_features(df).drop([col for col in df.columns if 'pm2.5' in col], axis=1, errors='ignore')

# Apply
train_enhanced = create_enhanced_features(train_clean)
test_enhanced = create_test_features_enhanced(test_clean)

# Feature alignment (now ~45 features)
common_features = [col for col in train_enhanced.columns if col in test_enhanced.columns and col not in ['pm2.5', 'No']]
X_train_common = train_enhanced[common_features]
y_train = train_enhanced['pm2.5']
X_test_common = test_enhanced[common_features]

print(f"📊 Enhanced features: {len(common_features)} (added EMAs + interactions)")



✅ Libraries imported - Enhanced for Experiment 7!
